In [1]:
%cd /home/pepa/PycharmProjects/claim-rank
!export PYTHONPATH=.
%load_ext autoreload
%autoreload 2

/home/pepa/PycharmProjects/claim-rank


In [2]:
import numpy as np
np.random.seed(42) # ! before importing keras!
from src.data.debates import read_debates, Debate
from src.features.feature_sets import get_cb_pipeline
from keras.layers import Input, Embedding, Conv1D, Dense
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from src.stats.rank_metrics import average_precision, precision_at_n

Using TensorFlow backend.


In [3]:
train_sentences = read_debates(Debate.FIRST) + read_debates(Debate.VP)
val_sentences = read_debates(Debate.SECOND)
test_sentences = read_debates(Debate.THIRD)

pipeline = get_cb_pipeline(train=train_sentences)
X_train = pipeline.fit_transform(train_sentences)
X_val = pipeline.transform(val_sentences)
X_test = pipeline.transform(test_sentences)

y_train_pf = np.array([s.labels[5] for s in train_sentences])
y_train_all = np.array([1 if s.label>0 else 0 for s in train_sentences])
y_train_wp = np.array([s.labels[4] for s in train_sentences])

y_val_pf = np.array([s.labels[5] for s in val_sentences])
y_val_all = np.array([1 if s.label>0 else 0 for s in val_sentences])
y_val_wp = np.array([s.labels[4] for s in val_sentences])

y_test_pf = np.array([s.labels[5] for s in test_sentences])
y_test_all = np.array([1 if s.label>0 else 0 for s in test_sentences])
y_test_wp = np.array([s.labels[4] for s in test_sentences])

['sent', 'tokens_num', 'text_len', 'bag_tfidf', 'ner', 'pos']


In [4]:
print(X_train.shape)
print(train_sentences[103].labels)
print(train_sentences[103].label)
print(X_train[0].shape)
print(np.array(y_train_all).shape)

(2761, 1503)
['0', '0', '1', '0', '0', '0', '0', '0', '0']
1
(1503,)
(2761,)


In [ ]:
# https://keras.io/getting-started/functional-api-guide/
# all and pf
input_layer = Input(shape=(X_train.shape[1],), name='input')
dense = Dense(10, name='dense')(input_layer)

pred_all = Dense(1, activation='sigmoid', name='pred_all')(dense)
pred_pf = Dense(1, activation='sigmoid', name='pred_pf')(dense)

model = Model(inputs=[input_layer], outputs=[pred_all, pred_pf])

model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y=[y_train_all, y_train_pf],validation_data=(X_val, [y_val_all, y_val_pf]), epochs=100)

In [ ]:
#  all, pf, and wp
input_layer = Input(shape=(X_train.shape[1],), name='input')
dense = Dense(10, name='dense')(input_layer)

pred_all = Dense(1, activation='sigmoid', name='pred_all')(dense)
pred_pf = Dense(1, activation='sigmoid', name='pred_pf')(dense)
pred_wp = Dense(1, activation='sigmoid', name='pred_wp')(dense)

model = Model(inputs=[input_layer], outputs=[pred_all, pred_pf, pred_wp])

model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y=[y_train_all, y_train_pf, y_train_wp],
          validation_data=(X_val, [y_val_all, y_val_pf, y_val_wp]), epochs=100)

In [ ]:
# only one task - all
input_layer = Input(shape=(X_train.shape[1],), name='input')
dense = Dense(10, name='dense')(input_layer)

pred_all = Dense(1, activation='sigmoid', name='pred_all')(dense)

model = Model(inputs=[input_layer], outputs=pred_all)

model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y=y_train_all,
          validation_data=(X_val, y_val_all), epochs=100)

In [ ]:
# only one task - pf
input_layer = Input(shape=(X_train.shape[1],), name='input')
dense = Dense(10, name='dense')(input_layer)

pred_pf = Dense(1, activation='sigmoid', name='pred_pf')(dense)

model = Model(inputs=[input_layer], outputs=pred_pf)

model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y=y_train_pf,
          validation_data=(X_val, y_val_pf), epochs=100)

In [ ]:
# TODO LSTM

In [ ]:
# TODO all feats

In [8]:
# get precision at 50! or avgp
#  all, pf, and wp
input_layer = Input(shape=(X_train.shape[1],), name='input')
hidden1 = Dense(1000, name='dense1', activation='relu')(input_layer)
drop1 = Dropout(rate=0.2,name='drop1')(hidden1)
hidden2 = Dense(100, name='dense2', activation='relu')(drop1)
drop2 = Dropout(rate=0.2,name='drop2')(hidden1)


pred_all = Dense(1, activation='relu', name='pred_all')(drop2)
pred_0 = Dense(1, activation='relu', name='pred_0')(drop2) 
pred_1 = Dense(1, activation='relu', name='pred_1')(drop2)
pred_2 = Dense(1, activation='relu', name='pred_2')(drop2)
pred_3 = Dense(1, activation='relu', name='pred_3')(drop2)
pred_4 = Dense(1, activation='relu', name='pred_4')(drop2)
pred_5 = Dense(1, activation='relu', name='pred_5')(drop2)
pred_6 = Dense(1, activation='relu', name='pred_6')(drop2)


y_train_0 = np.array([s.labels[0] for s in train_sentences])
y_train_1 = np.array([s.labels[1] for s in train_sentences])
y_train_2 = np.array([s.labels[2] for s in train_sentences])
y_train_3 = np.array([s.labels[3] for s in train_sentences])
y_train_4 = np.array([s.labels[4] for s in train_sentences])
y_train_5 = np.array([s.labels[5] for s in train_sentences])
y_train_6 = np.array([s.labels[6] for s in train_sentences])
y_val_0 = np.array([s.labels[0] for s in val_sentences])
y_val_1 = np.array([s.labels[1] for s in val_sentences])
y_val_2 = np.array([s.labels[2] for s in val_sentences])
y_val_3 = np.array([s.labels[3] for s in val_sentences])
y_val_4 = np.array([s.labels[4] for s in val_sentences])
y_val_5 = np.array([s.labels[5] for s in val_sentences])
y_val_6 = np.array([s.labels[6] for s in val_sentences])



model = Model(inputs=[input_layer], outputs=[pred_all, pred_0, pred_1, pred_2, pred_3, 
                                             pred_4, pred_5, pred_6])

model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y=[y_train_all, y_train_0, y_train_1, y_train_2, y_train_3, y_train_4, y_train_5, y_train_6],
          validation_data=(X_val, [y_val_all, y_val_0, y_val_1, y_val_2, y_val_3, 
                                   y_val_4, y_val_5, y_val_6]), epochs=30)

Train on 2761 samples, validate on 1303 samples
Epoch 1/30
2761/2761 [==============================] - 3s - loss: 4.6774 - pred_all_loss: 0.7242 - pred_0_loss: 0.3484 - pred_1_loss: 0.5256 - pred_2_loss: 0.4671 - pred_3_loss: 0.2817 - pred_4_loss: 1.1208 - pred_5_loss: 0.8062 - pred_6_loss: 0.4035 - pred_all_acc: 0.8254 - pred_0_acc: 0.9779 - pred_1_acc: 0.9674 - pred_2_acc: 0.9710 - pred_3_acc: 0.9823 - pred_4_acc: 0.9290 - pred_5_acc: 0.9500 - pred_6_acc: 0.9750 - val_loss: 4.8116 - val_pred_all_loss: 0.6653 - val_pred_0_loss: 0.3835 - val_pred_1_loss: 0.3587 - val_pred_2_loss: 0.4577 - val_pred_3_loss: 0.4082 - val_pred_4_loss: 1.1156 - val_pred_5_loss: 0.7422 - val_pred_6_loss: 0.6803 - val_pred_all_acc: 0.8396 - val_pred_0_acc: 0.9762 - val_pred_1_acc: 0.9777 - val_pred_2_acc: 0.9716 - val_pred_3_acc: 0.9747 - val_pred_4_acc: 0.9302 - val_pred_5_acc: 0.9540 - val_pred_6_acc: 0.9578

In [9]:
predicted_train = model.predict(X_train)
for pred, sentence in zip(predicted_train[0], train_sentences):
    sentence.pred = pred 
print(average_precision(train_sentences,1))
print(precision_at_n(train_sentences, n=50, agreement=1))

0.9337501029134335
1.0


In [10]:
predicted_val = model.predict(X_val)
for pred, sentence in zip(predicted_val[0], val_sentences):
    sentence.pred = pred 
print(average_precision(val_sentences,1))
print(precision_at_n(val_sentences, n=50, agreement=1))

0.2473886892756767
0.38


In [15]:
# get precision at 50! or avgp
#  all, pf, and wp
input_layer = Input(shape=(X_train.shape[1],), name='input')
hidden1 = Dense(1000, name='dense1', activation='relu')(input_layer)
drop1 = Dropout(rate=0.2,name='drop1')(hidden1)
hidden2 = Dense(100, name='dense2', activation='relu')(drop1)

pred_all = Dense(1, activation='relu', name='pred_all')(hidden2)
pred_5 = Dense(1, activation='relu', name='pred_5')(hidden2)

model = Model(inputs=[input_layer], outputs=[pred_all, pred_5])

model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y=[y_train_all, y_train_5],
          validation_data=(X_val, [y_val_all, y_val_5]), epochs=30)

Train on 2761 samples, validate on 1303 samples
Epoch 1/30
2761/2761 [==============================] - 2s - loss: 1.4240 - pred_all_loss: 0.6223 - pred_5_loss: 0.8017 - pred_all_acc: 0.8316 - pred_5_acc: 0.9500 - val_loss: 1.1466 - val_pred_all_loss: 0.4044 - val_pred_5_loss: 0.7422 - val_pred_all_acc: 0.8526 - val_pred_5_acc: 0.9540

In [16]:
predicted_train = model.predict(X_train)
for pred, sentence in zip(predicted_train[0], train_sentences):
    sentence.pred = pred 
print(average_precision(train_sentences,1))
print(precision_at_n(train_sentences, n=50, agreement=1))

predicted_val = model.predict(X_val)
for pred, sentence in zip(predicted_val[0], val_sentences):
    sentence.pred = pred 
print(average_precision(val_sentences,1))
print(precision_at_n(val_sentences, n=50, agreement=1))


0.9405339685797496
0.98
0.23564094973104566
0.36


In [17]:
predicted_train = model.predict(X_train)
for pred, sentence in zip(predicted_train[1], train_sentences):
    sentence.pred = pred 
print(average_precision(train_sentences,1))
print(precision_at_n(train_sentences, n=50, agreement=1))

predicted_val = model.predict(X_val)
for pred, sentence in zip(predicted_val[1], val_sentences):
    sentence.pred = pred 
print(average_precision(val_sentences,1))
print(precision_at_n(val_sentences, n=50, agreement=1))

0.45471170954005885
1.0
0.19700856024735783
0.32


In [19]:
# get precision at 50! or avgp
#  all, pf, and wp
input_layer = Input(shape=(X_train.shape[1],), name='input')
hidden1 = Dense(1000, name='dense1', activation='relu')(input_layer)
drop1 = Dropout(rate=0.2,name='drop1')(hidden1)
hidden2 = Dense(100, name='dense2', activation='relu')(drop1)

pred_all = Dense(1, activation='relu', name='pred_all')(hidden2)

model = Model(inputs=[input_layer], outputs=[pred_all])

model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y=[y_train_all],
          validation_data=(X_val, [y_val_all]), epochs=30)

predicted_train = model.predict(X_train)
for pred, sentence in zip(predicted_train[0], train_sentences):
    sentence.pred = pred 
print(average_precision(train_sentences,1))
print(precision_at_n(train_sentences, n=50, agreement=1))

predicted_val = model.predict(X_val)
for pred, sentence in zip(predicted_val[0], val_sentences):
    sentence.pred = pred 
print(average_precision(val_sentences,1))
print(precision_at_n(val_sentences, n=50, agreement=1))

Train on 2761 samples, validate on 1303 samples
Epoch 1/30
2761/2761 [==============================] - 2s - loss: 0.8153 - acc: 0.8233 - val_loss: 0.4742 - val_acc: 0.8427

In [ ]:
# get precision at 50! or avgp
#  all, pf, and wp
input_layer = Input(shape=(X_train.shape[1],), name='input')
hidden1 = Dense(1000, name='dense1', activation='relu')(input_layer)
drop1 = Dropout(rate=0.2,name='drop1')(hidden1)
hidden2 = Dense(100, name='dense2', activation='relu')(drop1)

pred_all = Dense(1, activation='relu', name='pred_all')(hidden2)

model = Model(inputs=[input_layer], outputs=[pred_all])

model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y=[y_train_all],
          validation_data=(X_val, [y_val_all]), epochs=30)

predicted_train = model.predict(X_train)
for pred, sentence in zip(predicted_train[0], train_sentences):
    sentence.pred = pred 
print(average_precision(train_sentences,1))
print(precision_at_n(train_sentences, n=50, agreement=1))

predicted_val = model.predict(X_val)
for pred, sentence in zip(predicted_val[0], val_sentences):
    sentence.pred = pred 
print(average_precision(val_sentences,1))
print(precision_at_n(val_sentences, n=50, agreement=1))